In [28]:
import datetime as dt
from dateutil import parser
import plotly.graph_objects as go
import pandas as pd
import sys
sys.path.append("../")

In [29]:
from api.oanda_api import OandaApi
from infrastructure.instrument_collection import InstrumentCollection as ic
from db.db import DataBaseMongo

In [30]:
database = DataBaseMongo()
icol = ic()
api = OandaApi()

In [31]:
load_icol = icol.DB_LoadInstruments()

In [32]:
pairs = []
for pair, val in icol.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [43]:
pairs[0:5]

['EUR_GBP', 'GBP_NZD', 'GBP_PLN', 'GBP_CHF', 'GBP_CAD']

In [34]:
data = []
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name=p, granularity='D', count=700)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_o) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

EUR_GBP
GBP_NZD
GBP_PLN
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_JPY
GBP_AUD
GBP_SGD
GBP_USD


In [42]:
candles_df.tail()[0:5]

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
7684,2024-02-05,107641,1.25368,1.26034,1.25322,1.25978,1.25301,1.26026,1.25279,1.25968,1.25436,1.26041,1.25337,1.25989,0.486568,GBP_USD
7685,2024-02-06,103924,1.25980,1.26422,1.25945,1.26284,1.25926,1.26413,1.25873,1.26274,1.26033,1.26431,1.25975,1.26293,0.241308,GBP_USD
7686,2024-02-07,103776,1.26260,1.26390,1.25718,1.26184,1.26197,1.26382,1.25709,1.26173,1.26322,1.26398,1.25727,1.26196,-0.060193,GBP_USD
7687,2024-02-08,93910,1.26214,1.26432,1.25990,1.26302,1.26129,1.26424,1.25981,1.26294,1.26298,1.26439,1.25998,1.26311,0.069723,GBP_USD
7688,2024-02-11,79553,1.26269,1.26548,1.26062,1.26282,1.26209,1.26538,1.26053,1.26271,1.26329,1.26558,1.26070,1.26294,0.010295,GBP_USD


In [38]:
calendar_data = database.query_all(DataBaseMongo.FOREX_CALANDER)
calendar_data_df = pd.DataFrame.from_dict(calendar_data)
calendar_data_df['date'] = pd.to_datetime(
    calendar_data_df['date']).dt.date
calendar_data_df_uk = calendar_data_df[calendar_data_df.country ==

                                       'united kingdom']

In [ ]:
calendar_data_df_uk[0:5]

,date,country,category,event,symbol,actual,previous,forecast
11,2020-01-13,united kingdom,balance of trade,balance of trade,UKTBTTBA,£4.03B,£-1.34B,£-3.2B
16,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.3%,1.5%,1.5%
20,2020-01-21,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,14.9K,14.9K,26K
34,2020-01-31,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9,-11,-10
38,2020-01-31,united kingdom,calendar,brexit deadline,,,,
...,...,...,...,...,...,...,...,...
2196,2023-12-12,united kingdom,unemployment rate,unemployment rate,UKUEILOR,4.2%,4.2%,4.3%
2203,2023-12-13,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.3%,0.2%,0.0%
2208,2023-12-14,united kingdom,interest rate,boe interest rate decision,UKBRBASE,5.25%,5.25%,5.25%
2222,2023-12-20,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,3.9%,4.6%,4.3%
